**Делаем сетевой анализ персонажей, где метриками выступают:**

* Популярность - как часто персонаж упоминается рядом с другими

* Связующая роль - кто соединяет разные группы героев

* Близость к центру - кто быстрее всех "дотянется" до любого другого героя

In [1]:
import pandas as pd
import networkx as nx
from itertools import combinations
import re
import pickle

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

C:\Users\Huawei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
with open('processed_data.pkl', 'rb') as f:
    base_data = pickle.load(f)

segments = base_data['segments']
my_mapping = base_data['my_mapping']
book1_end = base_data['book1_end']
book2_end = base_data['book2_end']

In [3]:
def get_network_metrics(segments, entity_mapping, window_size=50):
    G = nx.Graph()
    target_entities = set(entity_mapping.values())
    
    # Сбор связей
    for text in segments:
        words = re.sub(r'[^а-яё\_\s]', ' ', text.lower()).split()
        for i in range(len(words) - window_size + 1):
            window = words[i : i + window_size]
            found = list(set([word for word in window if word in target_entities]))
            if len(found) > 1:
                for char_a, char_b in combinations(sorted(found), 2):
                    if G.has_edge(char_a, char_b):
                        G[char_a][char_b]['weight'] += 1
                    else:
                        G.add_edge(char_a, char_b, weight=1)

    # Степень центральности
    degree = nx.degree_centrality(G)
    # Посредничество
    betweenness = nx.betweenness_centrality(G, weight='weight')
    # Близость
    closeness = nx.closeness_centrality(G)

    # Собираем в таблицу
    metrics_df = pd.DataFrame({
        'Персонаж': list(degree.keys()),
        'Популярность': list(degree.values()),
        'Связующая роль': list(betweenness.values()),
        'Близость к центру': list(closeness.values())
    })
    
    return metrics_df.sort_values(by='Связующая роль', ascending=False), G

df_network, G = get_network_metrics(segments, my_mapping)

In [4]:
# Создаем черный список для локаций и нарицательных имен
blacklist = [
    'средиземье', 'хоббитания', 'лихолесье', 'осгилиат', 'пригорье',
    'мордор', 'ривенделл', 'шир', 'рохан', 'гондор', 'арнор', 'раздол',
    'минас_тирит', 'изенгард', 'лориэн', 'мория', 'хоббит', 'ристания', 'том'
]

df_final = df_network[~df_network['Персонаж'].isin(blacklist)].copy()
df_final = df_final.sort_values(by='Связующая роль', ascending=False).head(20)

styled_table = df_final.style \
    .background_gradient(cmap='Blues', subset=['Популярность']) \
    .background_gradient(cmap='Oranges', subset=['Связующая роль']) \
    .format({
        'Популярность': '{:.3f}',
        'Связующая роль': '{:.3f}',
        'Близость к центру': '{:.3f}'
    }) \
    .set_caption("Сетевые метрики персонажей: Популярность vs Влияние")

styled_table

,Персонаж,Популярность,Связующая роль,Близость к центру
26,саурон,0.275,0.229,0.576
19,пин,0.475,0.187,0.650
52,лучиэнь,0.044,0.155,0.424
9,сэм,0.500,0.111,0.664
16,горлум,0.194,0.110,0.544
49,элронд,0.306,0.109,0.586
0,бильбо,0.356,0.086,0.597
61,балин,0.088,0.082,0.495
62,гимли,0.419,0.075,0.632
99,эомер,0.294,0.066,0.578


Смотрим на важность героев по первой книге

In [6]:
# Разбиваем сегменты на три книги
segments_b1 = segments[:book1_end]
segments_b2 = segments[book1_end:book2_end]
segments_b3 = segments[book2_end:]

# Считаем метрики для каждой книги
df_1 = get_network_metrics(segments_b1, my_mapping)[0].set_index('Персонаж')
df_2 = get_network_metrics(segments_b2, my_mapping)[0].set_index('Персонаж')
df_3 = get_network_metrics(segments_b3, my_mapping)[0].set_index('Персонаж')

# Объединяем в одну таблицу, чтобы сравнить динамику "Связующей роли"
comparison_df = pd.DataFrame({
    'Хранители': df_1['Связующая роль'],
    'Две твердыни': df_2['Связующая роль'],
    'Возвращение государя': df_3['Связующая роль']
}).fillna(0) # Если героя нет в книге, ставим 0

# Фильтруем (убираем локации и оставляем только топ героев)
comparison_clean = comparison_df[~comparison_df.index.isin(blacklist)]
comparison_clean = comparison_clean.sort_values(by='Хранители', ascending=False).head(15)
comparison_clean.style.background_gradient(cmap='Oranges', axis=1).format("{:.3f}")

,Хранители,Две твердыни,Возвращение государя
Персонаж,,,
сэм,0.278,0.108,0.019
бильбо,0.244,0.000,0.012
балин,0.220,0.000,0.000
накручинс,0.111,0.000,0.001
фродо,0.101,0.069,0.087
андуин,0.099,0.000,0.015
лучиэнь,0.098,0.000,0.000
пескунс,0.088,0.000,0.000
горислав,0.087,0.000,0.001


Смотрим, как меняется роль ключевых героев в каждой книге

In [7]:
# Создаем временную колонку с суммой влияния во всех книгах
comparison_df['Total_Influence'] = comparison_df.sum(axis=1)

# Сортируем по этой сумме, чтобы в ТОП попали герои, важные хотя бы в одной из книг
comparison_clean = comparison_df[~comparison_df.index.isin(blacklist)]
comparison_clean = comparison_clean.sort_values(by='Total_Influence', ascending=False).head(15)

# Удаляем временную колонку перед отрисовкой, чтобы она не портила таблицу
comparison_clean = comparison_clean.drop(columns=['Total_Influence'])

# Выводим стиль
comparison_clean.style.background_gradient(cmap='Oranges', axis=1).format("{:.3f}")

,Хранители,Две твердыни,Возвращение государя
Персонаж,,,
пин,0.076,0.129,0.287
сэм,0.278,0.108,0.019
элронд,0.026,0.297,0.041
саурон,0.064,0.000,0.210
фродо,0.101,0.069,0.087
бильбо,0.244,0.000,0.012
горлум,0.022,0.233,0.000
балин,0.220,0.000,0.000
леголас,0.055,0.150,0.015


In [8]:
final_results = {
    'global_metrics': df_final,
    'book_comparison': comparison_clean,
    'character_graph': G
}

with open('sna_final_results.pkl', 'wb') as f:
    pickle.dump(final_results, f)